In [1]:
from django.core.files import File
img = ImageFile()
with open('./images/02-NYH-underviser-AN-1.jpg', 'rb') as fp:
    imf=File(fp)
    img.source_file.save('foobar.jpg', imf)


In [2]:
from apps.photo.tasks import post_save_task

In [3]:
post_save_task(img)

23:21:02 2017-04-20 [ INFO] apps.photo.tasks     tasks.py:24   (post_save_task)
	autocrop



img.cropping

```
const closeCrop = (x, y, l, r, t, b, A) => {

const w = r - l
	const h = b - t
	const a = w / h
	const W = 0.5 * Math.min(A, 1, a > A ? w : h * A)
	const H = W / A
	const [X, Y] = [
		W * 2 > w ?
			[W, (l + r) / 2, 1 - W] :
			[l + W, x, r - W],
		H * 2 > h ?
			[H, (t + b) / 2, 1 - H] :
			[t + H, y, b - H],
	].map(triple => triple.sort((n, m) => n - m)[1])
	return {
		left: X - W,
		right: X + W,
		top: Y - H,
		bottom: Y + H,
	}
}
```

In [28]:
from apps.photo.cropping.boundingbox import Box
def closeCrop(x, y, left, right, top, bottom, aspect_ratio):
    l, r, t, b, A = left, right, top, bottom, aspect_ratio
    w, h = r - l, b - t
    a = w / h
    W = 0.5 * min(A, 1, w if a > A else h * A)
    H = W / A
    X, Y = [sorted(c)[1] for c in  (
        ((W, (l + r) / 2, 1 - W), (l + W, x, r - W))[W * 2 < w],
        ((H, (t + b) / 2, 1 - H), (t + H, y, b - H))[H * 2 < h]
    )]
    return Box(X - W, Y - H, X + W, Y + H)


def get_crop(image_file, width, height): 
    image_width, image_height = image_file.full_width, image_file.full_height
    aspect_ratio = (width * image_height) / (height * image_width)
    crop = closeCrop(**image_file.crop_box.serialize(), aspect_ratio=aspect_ratio)
    left, right = [int(val * image_width) for val in (crop.left, crop.right)]
    top, bottom = [int(val * image_height) for val in (crop.top, crop.bottom)]
    return Box(left, top, right, bottom)

print(img.crop_box, img.full_height, img.full_width)
height = 500
for width in [100, 200, 400, 1000]:
    crop = get_crop(img, width, height)
    print(crop, crop.ratio, width / height)

CropBox(x=0.61670000000000003, y=0.65663636363636368, left=0.2334, top=0.31327272727272726, right=1.0, bottom=1.0) 550 1000
Box(left=561, top=0, right=671, bottom=550) 0.2 0.2
Box(left=506, top=0, right=726, bottom=550) 0.4 0.4
Box(left=396, top=0, right=836, bottom=550) 0.8 0.8
Box(left=233, top=166, right=1000, bottom=550) 1.9973958333333333 2.0
